In [ ]:
#| default_exp inference.core
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.datasets.core import extract_mean_std_from_dls
from tsfast.data.loader import reset_model_state
from tsfast.models.layers import NormalizedModel
import numpy as np
import torch

In [ ]:
#| export

class InferenceWrapper:
    """
    A wrapper class to simplify inference with a trained tsfast/fastai Learner
    on NumPy data. Handles normalization and state reset automatically.
    """
    def __init__(self, learner,device='cpu'):
        """
        Initializes the inferencer.

        Args:
            learner: The trained tsfast/fastai Learner object.
            device = 'cpu': The device to run the inference on.
        """
        if not hasattr(learner, 'model') or not hasattr(learner, 'dls'):
            raise TypeError("Input 'learner' object does not appear to be a valid fastai/tsfast Learner.")

        self.device = device
        self.core_model = learner.model.to(self.device)

        # Extract normalization stats
        mean, std = extract_mean_std_from_dls(learner.dls)
        if mean is None or std is None:
             raise ValueError("Could not extract mean/std from learner's DataLoaders. Ensure normalization was used during training.")

        # Create and store the NormalizedModel
        # Assuming the normalization stats are for the combined input if fransys is used
        self.norm_model = NormalizedModel(self.core_model, mean, std).to(self.device)
        self.norm_model.eval() # Set to evaluation mode

    def inference(self, np_input: np.ndarray, np_output_init: np.ndarray = None) -> np.ndarray:
        """
        Performs inference on the input NumPy data.

        Args:
            np_input: The primary input data as a NumPy array.
                      Expected shapes: [seq_len] or [seq_len, features] or [1, seq_len, features].
            np_output_init: Optional secondary input data (e.g., for FranSys) as a NumPy array.
                            Expected shapes should be compatible with np_input after dimension expansion.

        Returns:
            The model's output as a NumPy array.
        """
        # Store original ndim for potential concatenation axis determination later
        original_ndim = np_input.ndim

        # Add batch and feature dimensions if needed
        if np_input.ndim == 1: # [seq_len] -> [1, seq_len, 1]
            np_input = np.expand_dims(np_input, axis=(0, -1))
            if np_output_init is not None:
                 if np_output_init.ndim == 1:
                     np_output_init = np.expand_dims(np_output_init, axis=(0, -1))
                 elif np_output_init.ndim == 2: # Handle [seq_len, features] case for init
                      np_output_init = np.expand_dims(np_output_init, axis=0)
                 elif np_output_init.ndim != 3:
                     raise ValueError(f"np_output_init shape {np_output_init.shape} incompatible with 1D np_input")

        elif np_input.ndim == 2: # [seq_len, features] -> [1, seq_len, features]
            np_input = np.expand_dims(np_input, axis=0)
            if np_output_init is not None:
                if np_output_init.ndim == 1: # Expand [seq_len] -> [1, seq_len, 1]
                    np_output_init = np.expand_dims(np_output_init, axis=(0, -1))
                elif np_output_init.ndim == 2: # Expand [seq_len, features] -> [1, seq_len, features]
                     np_output_init = np.expand_dims(np_output_init, axis=0)
                elif np_output_init.ndim != 3:
                     raise ValueError(f"np_output_init shape {np_output_init.shape} incompatible with 2D np_input")

        elif np_input.ndim == 3: # [1, seq_len, features] -> No change needed
            if np_input.shape[0] != 1:
                 raise ValueError(f"Input data with 3 dimensions should have batch size 1. Provided shape: {np_input.shape}")
            if np_output_init is not None:
                 if np_output_init.ndim == 1: # Expand [seq_len] -> [1, seq_len, 1]
                     np_output_init = np.expand_dims(np_output_init, axis=(0, -1))
                 elif np_output_init.ndim == 2: # Expand [seq_len, features] -> [1, seq_len, features]
                      np_output_init = np.expand_dims(np_output_init, axis=0)
                 elif np_output_init.ndim != 3:
                      raise ValueError(f"np_output_init shape {np_output_init.shape} incompatible with 3D np_input")
        else:
            raise ValueError(f"Input data should have 1, 2 or 3 dimensions. Provided shape: {np_input.shape}")

        # Concatenate inputs if np_output_init is provided
        if np_output_init is not None:
            # Ensure sequence lengths match
            if np_input.shape[1] != np_output_init.shape[1]:
                raise ValueError(f"Sequence lengths of np_input ({np_input.shape[1]}) and np_output_init ({np_output_init.shape[1]}) must match.")
            # Concatenate along the feature dimension (last axis)
            try:
                np_combined_input = np.concatenate((np_input, np_output_init), axis=-1)
            except ValueError as e:
                 raise ValueError(f"Could not concatenate inputs. Check shapes: np_input={np_input.shape}, np_output_init={np_output_init.shape}. Error: {e}")
            input_tensor = torch.from_numpy(np_combined_input).float().to(self.device)
        else:
            input_tensor = torch.from_numpy(np_input).float().to(self.device)


        output_tensor = None
        with torch.no_grad():
            reset_model_state(self.core_model) # Reset state before each inference call
            model_output = self.norm_model(input_tensor)

            # Handle tuple outputs (common in some RNNs)
            if isinstance(model_output, tuple):
                output_tensor = model_output[0]
            else:
                output_tensor = model_output

        if output_tensor is None:
            raise RuntimeError("Model did not return a valid output tensor.")

        # Remove batch dimension and return as NumPy array
        return output_tensor.squeeze(0).cpu().numpy()

    def __call__(self, np_input: np.ndarray, np_output_init: np.ndarray = None) -> np.ndarray:
        """Allows calling the predictor instance like a function."""
        return self.inference(np_input, np_output_init)

In [ ]:
from tsfast.datasets.core import create_dls_test
from tsfast.learner import RNNLearner
from tsfast.prediction import FranSysLearner


In [ ]:
dls = create_dls_test()
lrn = RNNLearner(dls)
model = InferenceWrapper(lrn)

In [ ]:
model(np.random.randn(100, 1)).shape

(100, 1)

In [ ]:
model(np.random.randn(100)).shape

(100, 1)

In [ ]:
model(np.random.randn(1,100,1)).shape

(100, 1)

In [ ]:
lrn = FranSysLearner(dls,10,attach_output=True)
model = InferenceWrapper(lrn)

In [ ]:
model(np.random.randn(100, 1),np.random.randn(100, 1)).shape

(100, 1)

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()